# Momentum

We've now introduced regularization, which allows us to get the overfitting problem down. That is, our test error and training error are similar.  This is good!  It's a lot better than what we had before.

However, training accuracy is now way down.  Recall that it's very unlikely that test accuracy is going to be better than training accuracy.  So we need a way to keep the training accuracy higher without overfitting.

One way to do this is to improve gradient descent through **momentum**.  This is something you should do anyway, as it makes gradient descent more accurate - we just didn't have much need for it when our training accuracy was already super high.

## How it Works

We fix a momentum parameter $\eta$, which is between 0 and 1.  Zero corresponds to no momentum (which defeats the purpose) and one corresponds to infinite momentum (a bad idea).  We'll talk about how to pick momentum in a little while.

The idea is, each training step, we have a vector $v$, representing the change to our weights due to gradient descent, that we're adding to our weights. With momentum, we actually add $v+\eta v_0$, where $v_0$ is the previous velocity.

What this means is that if two consecutive mini-batches disagree on what the ideal direction is, then the weights tend to move in the *average* of their votes, but with the combined speed of how much they agree.  To see why this is a good idea, suppose our objective function looks like a valley with very steep sides, but a very shallow downward slope in the middle (this turns out to be fairly common).  We're trying to find the lowest point of the valley.

If we start on one of the sides (again, fairly common) then a basic gradient descent step will launch us well up onto the other side, and only slightly downhill in our actual objective terms.  Then the next step will launch us back up the original side, and again only slightly downhill.  This process repeats and all the motion is wasted, taking a huge number of steps to get to the bottom.

Now if we use momentum, then that second step will be added to the first step.  We'll go slightly down the side toward the valley, but not too much, since the side-to-side motion will almost cancel out.  On the other hand, we'll almost double the speed at which we actually go down the valley, since those parts of the motion agree, so we make twice as much actual progress.  Even better, the third step has a little more side-to-side motion, but again the "downhill" part of the motion adds to the momentum, and we accelerate in the actual direction we want to go, since all these mini-batches agree.

**Note:** we *do* use momentum on regularization (and whatever else we end up using); momentum is a technique for improving and speeding up gradient descent, regardless of what the cost function is, while regularization (the way we've discussed) is just changing the cost function by penalizing large weights.

## Mathematical Meaning of Momentum Parameters

Because of the additivity of the process, you could see that with a specific momentum parameter $\eta$, a specific vector $v$ actually contributes $v+\eta v+\eta^2 v+\eta^3 v+\cdots$ to the weights, given enough epochs.  If $\eta=1$, this means we just keep adding $v$ forever, and it never degrades. This is bad -- we would never forget the effect of any particular mini-batch, no matter how irrelevant it has become.

However, if $0\leq \eta < 1$, then the total contribution of $v$ is $\sum_{i=0}^\infty \eta^i v$, which turns out to be $\frac{1}{1-\eta}v$.  So if $\eta=0.5$, we end up adding $2v$.  If $\eta=0.8$, we end up adding $5v$.  If $\eta=0.9$, we end up adding $10v$.  If $\eta=0.99$, we end up adding $100v$.  You can see the pattern.

Also, this series converges at an *exponential rate*.  So even though you need an "infinite number of training steps" to see the figures above, which you would obviously never have, it turns out that you get most of its contribution pretty quickly.  How quickly depends on $\eta$ -- the exact formula is if you want $\alpha$ of the contribution (e.g. $\alpha=0.90$ for 90%), then you need $$n=\frac{\ln(1-\alpha)}{\ln(\eta)}-1$$ which is probably not super illuminating.

However, $n$ increases with $\eta$ and $\alpha$, so assuming $\eta\leq 0.99$, you can always get 95% of the total contribution in 300 steps, which is less than a third of an epoch as we've worked with so far.  At that point $v$ is essentially gone.

If $\eta\leq 0.95$, you can get 95% of the total contribution in only 60 steps, which is almost immediate.  Again, at that point $v$ is essentially gone.

So we can think of $\eta$ as choosing *how long each particular contribution stays around*, understanding that it's a relative thing, and we can see the above formula as a way of measuring that.  If you like, we can reverse it, and say that if we want to accomplish 95% of the contribution in $n$ steps, what should $\eta$ be?  It turns out to be $$(0.05)^{1/(n+1)}$$

## Rules of Thumb

Maybe that's too much math for you, and maybe it isn't, but the question is: what levels of momentum are best?

In my experiments, somewhere between 0.8 and 0.9 seem best.  Researchers in the field, who know better than I do, tend to favor 0.9 or even higher (using tricks we'll talk about later, like dynamically updating momentum as the simulation progresses).  We'll look at a few different values, but we'll look at those primarily.

Just remember that 0.9 is "twice as much momentum" as 0.8, and 0.95 is "twice as much momentum" as 0.9, and so on.

## The Code

Implementing momentum is a very small change to our simulation code; we just need to track the velocities as we go, and instead of adding the gradient directly, we'll add the velocity (again, times a learning rate).  The actual changes are as follows:

1. Line `7` -- adding a parameter for momentum
2. Lines `19` and `20` -- keeping track of old velocities
3. Lines `39`, and `44` -- updating the velocities
4. Lines `40` and `45` -- using velocities instead of direct gradients

The rest of the code is unchanged.  Note that we only apply the learning rate when we actually add the velocity to the weights/biases.  It is possible, when implementing this yourself, to accidentally apply the learning rate to the velocity every timestep, which will almost certainly ruin the momentum and possibly blow up your simulation.

In [1]:
from basic_nn import *
import time

def run_exp(act_fn, cost_fn, init_fn, learning_rate,
            neuron_sizes, num_epochs, batch_size,
            l1_cost=0, l2_cost=0,
            momentum=0
           ):
    np.random.seed(313) # for determinism
    
    # Step 2: initialize
    weights, biases = init_fn(n, k, neuron_sizes)
    acts = [act_fn for _ in range(0, len(weights))]
    acts[-1] = act_sigmoid # last one is always sigmoid
    
    # Step 3: train
    t1 = time.time()
    
    weight_velocities = [0 for _ in range(0, len(weights))]
    biases_velocities = [0 for _ in range(0, len(biases))]

    for epoch in range(0, num_epochs):
        # we'll keep track of the cost as we go
        total_cost = 0
        num_batches = 0

        for X_mb, Y_mb in get_mini_batches(batch_size, train_X, train_Y):
            x, z, y = forward_prop(weights, biases, acts, X_mb)

            bp_grad_w, bp_grad_b = back_prop(weights, biases, acts, cost_fn, X_mb, Y_mb, x, y, z)
            l1_grad_w = lasso_cost(l1_cost, weights, biases, diff=True)
            l2_grad_w = ridge_cost(l2_cost, weights, biases, diff=True)

            for i in range(0, len(weights)):
                weight_grad = bp_grad_w[i] / len(X_mb)
                weight_grad += l1_grad_w[i]
                weight_grad += l2_grad_w[i]
                
                weight_velocities[i] = weight_velocities[i] * momentum + weight_grad
                weights[i] -= weight_velocities[i] * learning_rate
                
                biases_grad = bp_grad_b[i] / len(X_mb)
                
                biases_velocities[i] = biases_velocities[i] * momentum + biases_grad
                biases[i] -= biases_velocities[i] * learning_rate

            total_cost += cost_fn(y[-1], Y_mb, aggregate=True)
            num_batches += 1

        cost = total_cost / num_batches # average cost
        print("Cost {2:0.7f} through epoch {0}; took {1:0.3f} seconds so far.".format(epoch, time.time()-t1, cost))
    
    return weights, biases, acts

## Testing it Out

Let's try our experiments from before, with the same regularization, but with some momentum. We'll still use the MNIST training data:

In [2]:
from mnist_import import get_mnist_nice

train_X, train_Y, test_X, test_Y = get_mnist_nice()

n = train_X.shape[1]
k = train_Y.shape[1]

Now, let's try a combination of L1 penalties, L2 penalties, and momentum, just to see how it works.  First, a baseline with no momentum.  Note that I've chosen the regularization parameters with some care, so as to reduce overfitting, and given it as many epochs as I can stand.

In [3]:
act_fn = act_tanh
cost_fn = cost_CE
init_fn = initialize_xavier_tanh

learning_rate = 2

neuron_sizes = [100, 100]
num_epochs = 20
batch_size = 50

l1_cost = 0.5 / len(train_X)
l2_cost = 0.75 / len(train_X)

momentum = 0.0

weights, biases, acts = run_exp(act_fn, cost_fn, init_fn, learning_rate,
                                neuron_sizes, num_epochs, batch_size,
                                l1_cost=l1_cost, l2_cost=0,
                                momentum=momentum)

    
# Step 4: evaluate; training and test error
_, _, y = forward_prop(weights, biases, acts, train_X)
success_rate = classification_success_rate(y[-1], train_Y)
print("After {0} epochs, got {1:0.3f}% classifications correct (train).".format(num_epochs, 100*success_rate))

# Get test error, too
_, _, y = forward_prop(weights, biases, acts, test_X)
success_rate = classification_success_rate(y[-1], test_Y)
print("After {0} epochs, got {1:0.3f}% classifications correct (test).".format(num_epochs, 100*success_rate))

Cost 0.6479678 through epoch 0; took 8.488 seconds so far.
Cost 0.3176793 through epoch 1; took 16.676 seconds so far.
Cost 0.2523066 through epoch 2; took 25.171 seconds so far.
Cost 0.2183188 through epoch 3; took 34.236 seconds so far.
Cost 0.1950166 through epoch 4; took 43.131 seconds so far.
Cost 0.1828838 through epoch 5; took 52.219 seconds so far.
Cost 0.1710414 through epoch 6; took 61.076 seconds so far.
Cost 0.1630855 through epoch 7; took 69.864 seconds so far.
Cost 0.1563853 through epoch 8; took 78.901 seconds so far.
Cost 0.1516577 through epoch 9; took 87.787 seconds so far.
Cost 0.1446623 through epoch 10; took 96.943 seconds so far.
Cost 0.1397478 through epoch 11; took 105.783 seconds so far.
Cost 0.1347650 through epoch 12; took 114.445 seconds so far.
Cost 0.1317939 through epoch 13; took 123.254 seconds so far.
Cost 0.1293344 through epoch 14; took 132.359 seconds so far.
Cost 0.1264562 through epoch 15; took 141.590 seconds so far.
Cost 0.1227290 through epoch 1

It's still overfitting a little.  But looking at the cost over time, we see that we need more epochs - it was still improving its fit, but I cut it off too soon.  But I'm not made of time!  Let's use momentum and see where that gets us:

In [4]:
act_fn = act_tanh
cost_fn = cost_CE
init_fn = initialize_xavier_tanh

learning_rate = 2

neuron_sizes = [100, 100]
num_epochs = 20
batch_size = 50

l1_cost = 0.25 / len(train_X)
l2_cost = 0.5 / len(train_X)

momentum = 0.8

weights, biases, acts = run_exp(act_fn, cost_fn, init_fn, learning_rate,
                                neuron_sizes, num_epochs, batch_size,
                                l1_cost=l1_cost, l2_cost=0,
                                momentum=momentum)

    
# Step 4: evaluate; training and test error
_, _, y = forward_prop(weights, biases, acts, train_X)
success_rate = classification_success_rate(y[-1], train_Y)
print("After {0} epochs, got {1:0.3f}% classifications correct (train).".format(num_epochs, 100*success_rate))

# Get test error, too
_, _, y = forward_prop(weights, biases, acts, test_X)
success_rate = classification_success_rate(y[-1], test_Y)
print("After {0} epochs, got {1:0.3f}% classifications correct (test).".format(num_epochs, 100*success_rate))

Cost 18.4946867 through epoch 0; took 8.597 seconds so far.
Cost 13.8327904 through epoch 1; took 16.988 seconds so far.
Cost 15.2064617 through epoch 2; took 25.129 seconds so far.
Cost 17.6591008 through epoch 3; took 33.854 seconds so far.
Cost 18.0691744 through epoch 4; took 42.451 seconds so far.
Cost 17.9902699 through epoch 5; took 51.172 seconds so far.
Cost 20.1233503 through epoch 6; took 59.467 seconds so far.
Cost 22.7116701 through epoch 7; took 67.992 seconds so far.
Cost 21.9033218 through epoch 8; took 77.113 seconds so far.
Cost 23.4956074 through epoch 9; took 86.110 seconds so far.
Cost 25.6857530 through epoch 10; took 93.695 seconds so far.
Cost 27.5069423 through epoch 11; took 101.286 seconds so far.
Cost 25.5207972 through epoch 12; took 108.981 seconds so far.
Cost 23.1113438 through epoch 13; took 117.029 seconds so far.
Cost 24.0973440 through epoch 14; took 124.610 seconds so far.
Cost 23.3455768 through epoch 15; took 132.192 seconds so far.
Cost 20.099379

What the heck?  The cost is insane!  Why won't it go down?

The problem is that **the learning rate is much too high**.  But why?  It worked before!

## The Relationship Between Learning Rate and Momentum

Remember that math section before?  Maybe you skimmed it, I don't know.  But let's think about it from another perspective.  If we've gone enough timesteps, then at our timestep, we're adding $$v_t+\eta v_{t-1}+\eta^2 v_{t-2}+ \eta^3 v_{t-3}+\cdots$$ If all the velocities are approximately of the same magnitude, then the total velocity we're adding up is approximately $$\sum_{i=0}^\infty \eta^i=\frac{1}{1-\eta}$$ That is, we're multiplying the learning rate by approximately $\frac{1}{1-\eta}$. If $\eta=0.8$, that's multiplying the effective learning rate by five, if $\eta=0.9$, that's multiplying the effective learning rate by ten.

Of course if the old velocities disagree a lot, we won't get this much increase, but the point is there.  We put the learning rate at 2, which was really as high as we could stand, but now it's accidentally gotten too high.

One fix is to divide the intended learning rate by this fraction (that is, multiply it by $1-\eta$), which could work.  The more honest answer is that this is an okay starting place, but you'll need to tweak it from there.

## Better Experiments

In [5]:
act_fn = act_tanh
cost_fn = cost_CE
init_fn = initialize_xavier_tanh

learning_rate = 2

neuron_sizes = [100, 100]
num_epochs = 20
batch_size = 50

l1_cost = 0.25 / len(train_X)
l2_cost = 0.5 / len(train_X)

momentum = 0.8
learning_rate *= 1-momentum

weights, biases, acts = run_exp(act_fn, cost_fn, init_fn, learning_rate,
                                neuron_sizes, num_epochs, batch_size,
                                l1_cost=l1_cost, l2_cost=0,
                                momentum=momentum)

    
# Step 4: evaluate; training and test error
_, _, y = forward_prop(weights, biases, acts, train_X)
success_rate = classification_success_rate(y[-1], train_Y)
print("After {0} epochs, got {1:0.3f}% classifications correct (train).".format(num_epochs, 100*success_rate))

# Get test error, too
_, _, y = forward_prop(weights, biases, acts, test_X)
success_rate = classification_success_rate(y[-1], test_Y)
print("After {0} epochs, got {1:0.3f}% classifications correct (test).".format(num_epochs, 100*success_rate))

Cost 0.7855278 through epoch 0; took 8.330 seconds so far.
Cost 0.2977677 through epoch 1; took 16.187 seconds so far.
Cost 0.2301322 through epoch 2; took 24.084 seconds so far.
Cost 0.1961107 through epoch 3; took 32.370 seconds so far.
Cost 0.1722877 through epoch 4; took 41.143 seconds so far.
Cost 0.1545653 through epoch 5; took 49.079 seconds so far.
Cost 0.1435691 through epoch 6; took 57.843 seconds so far.
Cost 0.1305939 through epoch 7; took 65.901 seconds so far.
Cost 0.1209087 through epoch 8; took 73.752 seconds so far.
Cost 0.1172330 through epoch 9; took 81.670 seconds so far.
Cost 0.1097976 through epoch 10; took 89.896 seconds so far.
Cost 0.1044769 through epoch 11; took 98.530 seconds so far.
Cost 0.0990088 through epoch 12; took 106.948 seconds so far.
Cost 0.0980855 through epoch 13; took 114.909 seconds so far.
Cost 0.0955043 through epoch 14; took 122.740 seconds so far.
Cost 0.0895981 through epoch 15; took 130.915 seconds so far.
Cost 0.0863248 through epoch 16

Observe that the cost is going down a little faster than it did in the original experiment, even though the effective learning rate is theoretically equal to the original, and practically speaking is probably lower.  This is because it's finding the effective directions to move along, rather than randomly moving around.

Let's try with even more momentum:

In [6]:
act_fn = act_tanh
cost_fn = cost_CE
init_fn = initialize_xavier_tanh

learning_rate = 2

neuron_sizes = [100, 100]
num_epochs = 20
batch_size = 50

l1_cost = 0.25 / len(train_X)
l2_cost = 0.5 / len(train_X)

momentum = 0.9
learning_rate *= 1-momentum

weights, biases, acts = run_exp(act_fn, cost_fn, init_fn, learning_rate,
                                neuron_sizes, num_epochs, batch_size,
                                l1_cost=l1_cost, l2_cost=0,
                                momentum=momentum)

    
# Step 4: evaluate; training and test error
_, _, y = forward_prop(weights, biases, acts, train_X)
success_rate = classification_success_rate(y[-1], train_Y)
print("After {0} epochs, got {1:0.3f}% classifications correct (train).".format(num_epochs, 100*success_rate))

# Get test error, too
_, _, y = forward_prop(weights, biases, acts, test_X)
success_rate = classification_success_rate(y[-1], test_Y)
print("After {0} epochs, got {1:0.3f}% classifications correct (test).".format(num_epochs, 100*success_rate))

Cost 4.5568776 through epoch 0; took 7.821 seconds so far.
Cost 0.3618658 through epoch 1; took 15.565 seconds so far.
Cost 0.2533338 through epoch 2; took 23.475 seconds so far.
Cost 0.2083368 through epoch 3; took 31.575 seconds so far.
Cost 0.1737058 through epoch 4; took 39.437 seconds so far.
Cost 0.1601622 through epoch 5; took 47.316 seconds so far.
Cost 0.1447033 through epoch 6; took 55.229 seconds so far.
Cost 0.1309208 through epoch 7; took 63.316 seconds so far.
Cost 0.1272872 through epoch 8; took 71.095 seconds so far.
Cost 0.1180604 through epoch 9; took 79.316 seconds so far.
Cost 0.1093092 through epoch 10; took 87.113 seconds so far.
Cost 0.1036043 through epoch 11; took 94.908 seconds so far.
Cost 0.1010903 through epoch 12; took 103.133 seconds so far.
Cost 0.0981105 through epoch 13; took 110.957 seconds so far.
Cost 0.0945673 through epoch 14; took 118.751 seconds so far.
Cost 0.0896110 through epoch 15; took 126.506 seconds so far.
Cost 0.0870564 through epoch 16

Better!  It eventually caught up and beat the other simulation, in terms of cost, training accuracy, and test accuracy (all slightly).  Let's try yet another one:

In [7]:
act_fn = act_tanh
cost_fn = cost_CE
init_fn = initialize_xavier_tanh

learning_rate = 2

neuron_sizes = [100, 100]
num_epochs = 20
batch_size = 50

l1_cost = 0.25 / len(train_X)
l2_cost = 0.5 / len(train_X)

momentum = 0.99
learning_rate *= 1-momentum

weights, biases, acts = run_exp(act_fn, cost_fn, init_fn, learning_rate,
                                neuron_sizes, num_epochs, batch_size,
                                l1_cost=l1_cost, l2_cost=0,
                                momentum=momentum)

    
# Step 4: evaluate; training and test error
_, _, y = forward_prop(weights, biases, acts, train_X)
success_rate = classification_success_rate(y[-1], train_Y)
print("After {0} epochs, got {1:0.3f}% classifications correct (train).".format(num_epochs, 100*success_rate))

# Get test error, too
_, _, y = forward_prop(weights, biases, acts, test_X)
success_rate = classification_success_rate(y[-1], test_Y)
print("After {0} epochs, got {1:0.3f}% classifications correct (test).".format(num_epochs, 100*success_rate))

Cost 32.7289564 through epoch 0; took 7.453 seconds so far.
Cost 35.2144408 through epoch 1; took 14.886 seconds so far.
Cost 34.5891639 through epoch 2; took 22.530 seconds so far.
Cost 33.6685610 through epoch 3; took 29.961 seconds so far.
Cost 32.5866875 through epoch 4; took 37.297 seconds so far.
Cost 31.3134711 through epoch 5; took 44.814 seconds so far.
Cost 29.8400733 through epoch 6; took 52.159 seconds so far.
Cost 28.0678613 through epoch 7; took 59.907 seconds so far.
Cost 25.8915033 through epoch 8; took 67.466 seconds so far.
Cost 23.3152292 through epoch 9; took 74.830 seconds so far.
Cost 20.4511033 through epoch 10; took 82.177 seconds so far.
Cost 17.5274746 through epoch 11; took 89.506 seconds so far.
Cost 14.7606059 through epoch 12; took 96.876 seconds so far.
Cost 12.1184489 through epoch 13; took 104.243 seconds so far.
Cost 2.9844576 through epoch 14; took 111.421 seconds so far.
Cost 0.4132006 through epoch 15; took 118.828 seconds so far.
Cost 0.2952190 thr

So, it's possible to go too far.  Still, despite the initial cost calamity, it did eventually figure out what it wanted and started to optimize.  This turns out to be a meaningful observsation, not a coincidence, and we'll talk about it later.

## TL;DR

If all that was too much and it's not clear what to take away from it, let me summarize with a few bullet points.
1. Momentum is a way of improving gradient descent, and it makes it work better.
2. Better is not the same as faster; momentum is not just "speeding up your search," although it can.
2. Momentum values between 0.8 and 0.9 are good; less than that is pointless and more than that is dangerous.
3. Momentum increases the effective learning rate, and you need to reduce the learning rate proportionally -- scale it by $1-\eta$.
5. That scaling factor is just a start, and it's not an excuse to skip your parameter search.